In [ ]:
import pandas as pd , numpy as np , matplotlib.pyplot as plt,tensorflow as tf , matplotlib as mpl
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.models import Model
import tensorflow.keras.layers as layers

In [ ]:
df = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv',header=None,prefix='c')
df.head()

In [ ]:
last_col = df.pop('c140')
df.insert(0,'target',last_col)

In [ ]:
df['target'].value_counts()

In [ ]:
df.describe()

In [ ]:
train,test,y_train,y_test = tts(df.values , df.values[:,0],test_size = 0.2 ,random_state = 111)

In [ ]:
scaler = MinMaxScaler()
scaler = scaler.fit(train)
train_sc = scaler.transform(train)
test_sc = scaler.transform(test)

In [ ]:
normal_train_df = pd.DataFrame(train_sc).add_prefix('c').query('c0==1').values[:,1:]
anomly_train_df = pd.DataFrame(train_sc).add_prefix('c').query('c0==0').values[:,1:]

In [ ]:
normal_test_df = pd.DataFrame(test_sc).add_prefix('c').query('c0==1').values[:,1:]
anomly_test_df = pd.DataFrame(test_sc).add_prefix('c').query('c0==0').values[:,1:]

In [ ]:
print(normal_train_df.shape)
print(anomly_train_df.shape)

In [ ]:
plt.plot(normal_train_df[0])
plt.plot(normal_train_df[1])
plt.plot(normal_train_df[2])

In [ ]:
plt.plot(anomly_train_df[0])
plt.plot(anomly_train_df[1])
plt.plot(anomly_train_df[2])

In [ ]:
import tensorflow.keras.layers as layers

class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(64, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


model = AnomalyDetector()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 2,mode= 'min')

model.compile(optimizer = 'adam',loss='mae')

In [ ]:
history = model.fit(normal_train_df,normal_train_df,epochs = 100,batch_size = 128,validation_data = (train_sc[:,1:],train_sc[:,1:]),shuffle=True)

In [ ]:
encoder_out = model.encoder(normal_test_df).numpy()
decoder_out = model.decoder(encoder_out).numpy()

In [ ]:
print(encoder_out.shape)
print(decoder_out.shape)

In [ ]:
plt.plot(normal_test_df[0],'b')
plt.plot(decoder_out[0],'r')
plt.fill_between(np.arange(140),decoder_out[0],normal_test_df[0],color='lightcoral')
# plt.fill_between(np.arange(140), decoded_imgs[0], normal_test_data[0], color='lightcoral' )

In [ ]:
encoder_out_a = model.encoder(anomly_test_df).numpy()
decoder_out_a = model.decoder(encoder_out_a).numpy()

plt.plot(anomly_test_df[0],'b')
plt.plot(decoder_out_a[0],'r')
plt.fill_between(np.arange(140),decoder_out_a[0],anomly_test_df[0],color = 'lightcoral')

In [ ]:
reconstruction= model.predict(normal_test_df)
train_loss  = tf.keras.losses.mae(reconstruction,normal_test_df)

plt.hist(train_loss,bins = 50)
plt.show()

In [ ]:
print(np.mean(train_loss))
print(np.std(train_loss))

In [ ]:
threshold = np.mean(train_loss)+2*np.std(train_loss)
threshold

In [ ]:
reconstruction_a = model.predict(anomly_test_df)
train_loss_a = tf.keras.losses.mae(reconstruction_a,anomly_test_df)

plt.hist(train_loss_a,bins=50)
plt.show()

In [ ]:
print(np.mean(train_loss_a))
print(np.std(train_loss_a))

In [ ]:
plt.hist(train_loss,bins=50,label='normal')
plt.hist(train_loss_a,bins=50,label='anomly')
plt.axvline(threshold , color='r',linewidth=3,linestyle = 'dashed',label = '{:03f}'.format(threshold))
plt.show()

In [ ]:
tf.math.less(train_loss,threshold)

In [ ]:
pred = tf.math.less(train_loss,threshold)
tf.math.count_nonzero(pred)

In [ ]:
pred.shape

In [ ]:
# for noraml use tf.math.less , for anomly use tf.math.greater than threshold
pred_a = tf.math.greater(train_loss_a,threshold)
tf.math.count_nonzero(pred_a)

In [ ]:
pred_a.shape

In [ ]:
plt.plot(train[0],color = 'r',linewidth = 2)
plt.axhline(threshold,color = 'b',linewidth = 1.5,linestyle = '--')

### anomly detection using LSTM

In [ ]:
import tensorflow as tf , tensorflow.keras as keras , numpy as np ,pandas as pd , matplotlib.pyplot as plt ,plotly.graph_objects as go,seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dense , Dropout , RepeatVector,TimeDistributed

In [ ]:
df = pd.read_csv('/content/GOOGL.csv')
df.head()

In [ ]:
df = df[['Date','Close']]
df.head()

In [ ]:
df.info()

In [ ]:
df['Date'].min(),df['Date'].max()

### visualize data

In [ ]:
plt.figure(figsize = (15,6))
plt.plot(df['Date'],df['Close'],label = 'closing price')
plt.title('Apple .Inc stock prices 2004 - 2020')
plt.show()

### data preprocesing

In [ ]:
train = df.loc[df['Date']<='2017-12-24']
test = df.loc[df['Date']>'2017-12-24']
train.shape,test.shape

In [ ]:
scaler = StandardScaler().fit(train['Close'].values.reshape(-1,1))
train['Close'] = scaler.transform(train['Close'].values.reshape(-1,1))
test['Close'] = scaler.transform(test['Close'].values.reshape(-1,1))    # or np.array(test['close].reshape(-1,1))

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(train['Close'])
plt.show()

In [ ]:
time_steps = 30
def create_seq(x,y,time_steps=time_steps):
  x_out,y_out = [],[]
  for i in range(len(x)-time_steps):
    x_out.append(x.iloc[i:(i+time_steps)].values)
    y_out.append(y.iloc[i+time_steps])
  return np.array(x_out) , np.array(y_out)

x_train , y_train = create_seq(train['Close'],train['Close'])
x_test,y_test = create_seq(test['Close'],test['Close'])
print(f'x_train shape is{x_train.shape}')
print(f'x_test shape is{x_test.shape}')

In [ ]:
model = Sequential([
    LSTM(128,activation='tanh',input_shape = (lag,n_fea)),
    Dropout(0.2),
    RepeatVector(lag),
    LSTM(128,activation='tanh',return_sequences=True),
    Dropout(0.2),
    TimeDistributed(Dense(n_fea))
])

model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001),loss='mae')
model.summary()

In [ ]:
history = model.fit(x_train,y_train,epochs =100,validation_data = (x_test,y_test),batch_size = 32,
                    callbacks = keras.callbacks.EarlyStopping(patience = 5,mode = 'min',monitor = 'val_loss'))